# Montar Drive + Imports

In [ ]:
import os
import pandas as pd
from google.colab import drive
import re

In [ ]:
# 1. Montar Google Drive
print("Paso 1: Montando Google Drive...")
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Máster - Data Science/TFM/Data/ESTUDIO_MULTIPLEX')

Paso 1: Montando Google Drive...
Mounted at /content/drive


# Crear listado pacientes (patients.txt)

In [ ]:
def listar_pacientes_y_origen(directorio_base=".", nombre_archivo_salida="patients.csv"):
    """
    Extrae la lista de valores de la columna 'id' de CLINIC_{origen}.csv
    dentro de DADES_{origen} para cada origen, y los escribe en un archivo CSV.

    Args:
        directorio_base (str): El directorio superior que contiene DADES_HCB y DADES_NAPLES.
        nombre_archivo_salida (str): El nombre del archivo de salida (por defecto 'patients.csv').
    """

    # 1. Definición de directorios y archivo clínico
    origenes = ["HCB", "NAPLES"]
    archivo_clinico = "CLINIC_{origen}.csv"

    # Conjunto para almacenar pares únicos (id_paciente, origen)
    pacientes_encontrados = set()

    print(f"Buscando IDs de paciente en los archivos clínicos dentro de DADES_HCB/ y DADES_NAPLES/...")

    # 2. Recorrer los orígenes
    for origen in origenes:
        ruta_origen = os.path.join(directorio_base, f"DADES_{origen}")
        nombre_archivo_clinico = archivo_clinico.format(origen=origen)
        ruta_archivo_clinico = os.path.join(ruta_origen, nombre_archivo_clinico)

        # Verificar si la carpeta de origen y el archivo clínico existen
        if not os.path.isdir(ruta_origen):
            print(f"⚠️ Advertencia: El directorio {ruta_origen} no se encontró. Saltando.")
            continue

        if not os.path.exists(ruta_archivo_clinico):
            print(f"⚠️ Advertencia: El archivo {ruta_archivo_clinico} no se encontró. Saltando.")
            continue

        # 3. Leer el archivo clínico y extraer la columna 'ID'
        print(f"Procesando {nombre_archivo_clinico}...")
        try:
            # Leer el CSV. Asegúrate de que el delimitador sea correcto si no es la coma estándar.
            df_clinico = pd.read_csv(ruta_archivo_clinico)

            if 'id' in df_clinico.columns:
                # Obtener la columna 'ID', convertir a una lista de strings y eliminar duplicados
                ids_pacientes = df_clinico['id'].astype(str).unique()

                # Añadir cada ID y su origen al conjunto de pacientes encontrados
                for id_paciente in ids_pacientes:
                    pacientes_encontrados.add((id_paciente, origen))

                print(f"   -> Encontrados {len(ids_pacientes)} IDs en {nombre_archivo_clinico}.")

            else:
                print(f"❌ Error: La columna 'id' no se encontró en {nombre_archivo_clinico}.")

        except Exception as e:
            print(f"❌ Error al leer o procesar {ruta_archivo_clinico}: {e}")

    # 4. Escribir los resultados en el archivo de salida CSV
    if pacientes_encontrados:
        try:
            with open(nombre_archivo_salida, 'w') as f:
                # Escribir una cabecera, usando COMA como delimitador para CSV
                f.write("id_paciente,origen\n")

                # Escribir cada par (ID, Origen) ordenado, usando COMA como separador
                for id_paciente, origen in sorted(list(pacientes_encontrados)):
                    f.write(f"{id_paciente},{origen}\n")

            print(f"✅ Proceso completado. Se listaron {len(pacientes_encontrados)} pacientes únicos en el archivo CSV '{nombre_archivo_salida}'.")

        except IOError as e:
            print(f"❌ Error al escribir en el archivo {nombre_archivo_salida}: {e}")
    else:
        print("🤔 No se encontraron IDs de paciente en los archivos clínicos especificados.")

# --- Ejecución del Script ---
# El archivo de salida será 'patients.csv'
listar_pacientes_y_origen()

Buscando IDs de paciente en los archivos clínicos dentro de DADES_HCB/ y DADES_NAPLES/...
Procesando CLINIC_HCB.csv...
   -> Encontrados 165 IDs en CLINIC_HCB.csv.
Procesando CLINIC_NAPLES.csv...
   -> Encontrados 105 IDs en CLINIC_NAPLES.csv.
✅ Proceso completado. Se listaron 270 pacientes únicos en el archivo CSV 'patients.csv'.
